### This notebook is an option to running the automated pipeline, e.g., for debugging possible problems in the pipeline-methods.

In [1]:
file = 'data/filtered-portugal-latest.osm.pbf'

In [2]:
#read the building data from the osm-file and insert into a geopandas dataframe
from pipeline.buildings import create_geodataframe
gdf = create_geodataframe(file)

Loading osm-data from file data/filtered-portugal-latest.osm.pbf, this will take a while...
invalid area (area_id=1219870002)
invalid area (area_id=31268923)
Creating geodataframe from the data from data/filtered-portugal-latest.osm.pbf, this will take a while...


In [3]:
#estimate levels for the buildings lacking level data
from pipeline.level_estimation import estimate_levels_with_randomforest
gdf = estimate_levels_with_randomforest(gdf)

Predicting levels, please wait...
Using training data length 14992
Fitting model to data, please wait...


In [4]:
#load the climate zones into a geopandas dataframe
from pipeline.climate_zones import get_climate_zones
zones = get_climate_zones()

In [5]:
#calculate energy demand using the climate zone data
from pipeline.energy_demand import get_energy_demand
gdf = get_energy_demand(gdf, zones)

Estimating energy demand per climate zone for country portugal, this will take a while...


In [6]:
#calculate co2-emissions using the co2-rate data
from pipeline.co2_emissions import get_co2_emissions
import pandas as pd
co2_rates = pd.read_csv('pipeline/co2rates.csv', index_col=0)
gdf = get_co2_emissions(gdf, co2_rates)

Calculating co2-emissions for country portugal...


In [17]:
#create kilometer square tiles of co2-emission sums
from pipeline.square_tiles import create_square_tiles
tuple = create_square_tiles(gdf)

Creating square-tiles of sum emission levels for portugal, this will take a while...


In [19]:
#create database connection
from dotenv import dotenv_values
cf = dotenv_values(".env")
db_url = f'postgresql://{cf["USER"]}:{cf["PW"]}@{cf["URL"]}:{cf["PORT"]}/{cf["DB"]}'

In [20]:
from sqlalchemy import create_engine
engine = create_engine(db_url)  

In [26]:
#insert the data into the postgis database
gdf.to_postgis('buildings', engine, if_exists="append") 
tuple[0].to_postgis('squares_co2', engine, if_exists="append") 
tuple[1].to_postgis('squares_opt_co2', engine, if_exists="append") 
tuple[2].to_postgis('squares_diff', engine, if_exists="append") 